In [2]:
!pip install -q git+https://github.com/huggingface/transformers@21fac7abba2a37fae86106f87fcf9974fd1e3830 accelerate
!pip install -q optimum
!pip install -q auto-gptq
!pip install -q qwen-vl-utils[decord]

In [3]:
!pip install optimum
!pip install auto-gptq

In [4]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

In [5]:

model_id = "/kaggle/input/qwen2-vl/transformers/qwen2-vl-2b-instruct-gptq-int4/1"
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id, torch_dtype="auto", device_map="auto"  # Ensure proper mode is selected

)
processor = AutoProcessor.from_pretrained(model_id)

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:4728: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [6]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                # "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
                "image": "https://www.akospices.com/cdn/shop/articles/Indian_Grocery_Store_in_UK_Online.jpg?v=1584438200",
                # "image": "https://sriexim.com/assets/images/groceries-sri-exim-4-578x434.jpeg",
                # "image": "/kaggle/input/images1/sankhe.jpg",
            },
            {"type": "text", "text": """This image contains fruits, vegetables, or packaged products.
                    Please analyze the image and provide:
                    - For products or fruits or vegetables:
                        - Product Name
                        - Product Category
                        - Product Quantity
                        - Product Count
                        - Expiry Date (if available)
                        - Freshness Index (based on visual cues)
                        - Estimated Shelf Life
                        If there are multiple products,give answer for each"""},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=512)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

["- Product Name: Annapurna Special Atta\n- Product Category: Baking\n- Product Quantity: 1\n- Product Count: 1\n- Expiry Date: Not visible\n- Freshness Index: Not visible\n- Estimated Shelf Life: Not visible\n- Product Name: Taj Mahal\n- Product Category: Cooking\n- Product Quantity: 1\n- Product Count: 1\n- Expiry Date: Not visible\n- Freshness Index: Not visible\n- Estimated Shelf Life: Not visible\n- Product Name: Frooti\n- Product Category: Beverages\n- Product Quantity: 1\n- Product Count: 1\n- Expiry Date: Not visible\n- Freshness Index: Not visible\n- Estimated Shelf Life: Not visible\n- Product Name: Tata salt\n- Product Category: Seasoning\n- Product Quantity: 1\n- Product Count: 1\n- Expiry Date: Not visible\n- Freshness Index: Not visible\n- Estimated Shelf Life: Not visible\n- Product Name: Ghee\n- Product Category: Cooking\n- Product Quantity: 1\n- Product Count: 1\n- Expiry Date: Not visible\n- Freshness Index: Not visible\n- Estimated Shelf Life: Not visible\n- Product 

In [10]:
# Save the model's state_dict (preferred for production)
model_file = "qwen2vl_model_state.pt"
torch.save(model.state_dict(), model_file)
print(f"Model state_dict saved to {model_file}")


AttributeError: Can't pickle local object 'GPTQQuantizer.post_init_model.<locals>.StoreAttr'